Створити RNN для семантичного аналізу тексту з використанням моделі LSTM;
Розробити власну архітектуру, кількість рекурентних шарів, функції
активації, вибрати оптимізатор.
Отримати по дві рецензії (позитивну і негативну).

In [18]:
# Завантаження бібліотек
import numpy as np
import pandas as pd
import re
from tensorflow.keras.layers import Dense, LSTM, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Завантаження датасету

Обрала датасет harry_potter_reviews.csv так як в ньому є вся необхідна інформація для аналізу. Обираю лише необхідні колонки для пришвидшення роботи.

In [19]:
# Завантаження та попередня обробка відгуків
hp_reviews = pd.read_csv("C:\\AI\\harry_potter_reviews.csv")
hp_reviews['sentiment'] = (hp_reviews['rating'] > 3).astype(int)
comments = hp_reviews['comment'].tolist()
sentiments = hp_reviews['sentiment'].tolist()
hp_reviews.head()

,user_id,user_sex,user_age,user_country,rating,comment,favourite_character,date,sentiment
0,0,female,50,Germany,2.5,"""The transitions between scenes were awkward, ...",Severus Snape,2004-12-27,0
1,1,female,23,Spain,4.0,"""Severus Snape's role adds an intriguing layer.""",Severus Snape,2003-11-22,1
2,2,male,32,France,3.0,"""The pacing was a bit slow, but the characters...",Ron Weasley,2005-09-16,0
3,3,female,24,Turkey,4.5,"""Hagrid's love for magical creatures is heartw...",Rubeus Hagrid,2002-09-17,1
4,4,female,40,Spain,5.0,"""Neville Longbottom's courage is awe-inspiring.""",Neville Longbottom,2004-10-17,1


# Токенізація

З огляду на послідовність символів і певну одиницю документа, токенізація - це завдання розбити її на частини, які називаються токенами, можливо, в той же час викинувши певні символи, такі як пунктуація. Токенізатор створює токени для кожного слова в корпусі даних і відображає їх в індекс.

In [20]:
# Токенізація і доповнення
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(comments)
data = tokenizer.texts_to_sequences(comments)
max_text_len = 60  
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)

[[  0   0   0 ...  14 195  41]
 [  0   0   0 ... 153   2 109]
 [  0   0   0 ...   7   4  80]
 ...
 [  0   0   0 ...  52   1 222]
 [  0   0   0 ... 104   4  80]
 [  0   0   0 ... 146 106 182]]


In [21]:
# Підготовка набору даних
X = np.array(data_pad)
Y = np.array([[1, 0] if sentiment == 1 else [0, 1] for sentiment in sentiments])
print(X, Y)

[[  0   0   0 ...  14 195  41]
 [  0   0   0 ... 153   2 109]
 [  0   0   0 ...   7   4  80]
 ...
 [  0   0   0 ...  52   1 222]
 [  0   0   0 ... 104   4  80]
 [  0   0   0 ... 146 106 182]] [[0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0

# Тренуємо модель

Спершу перемішую набори даних: створюється масив індексів для перемішування зразків у наборі даних X та відповідних міток Y, щоб забезпечити випадковий порядок даних перед тренуванням моделі. 

Спочатку текст перетворюється на вектори за допомогою шару Embedding. Потім застосовується Dropout для запобігання перенавчанню. Шар Conv1D допомагає виявити важливі ознаки у тексті, використовуючи фільтри та активацію ReLU. Послідовні шари LSTM обробляють послідовності даних, зберігаючи корисну інформацію та відкидаючи непотрібну, а додаткові шари Dropout мінімізують ризик перенавчання. В кінці моделі є Dense шари, які використовуються для класифікації виводу в одну з двох категорій. Всю модель компілюють із використанням крос-ентропії як функції втрати та оптимізатора Adam. Для покращення тренування я інтегрувала зворотний виклик ReduceLROnPlateau, який знижує швидкість навчання, коли модель перестає покращуватися, допомагаючи зберегти оптимальне навчання і уникнути застрягання на локальних мінімумах.

In [22]:
from tensorflow.keras.layers import Conv1D, Dropout, Bidirectional, LSTM, Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau

model = Sequential()
model.add(Embedding(maxWordsCount, 128))
model.add(Dropout(0.25))  # Dropout for input layer
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Implement ReduceLROnPlateau to adjust the learning rate
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

history = model.fit(X, Y, batch_size=32, epochs=40, validation_split=0.2, callbacks=[lr_reduction])


Epoch 1/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 79ms/step - accuracy: 0.7072 - loss: 0.6299 - val_accuracy: 0.7374 - val_loss: 0.4969 - learning_rate: 0.0010
Epoch 2/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.7828 - loss: 0.5243 - val_accuracy: 0.7374 - val_loss: 0.5007 - learning_rate: 0.0010
Epoch 3/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.7605 - loss: 0.5312 - val_accuracy: 0.7374 - val_loss: 0.4614 - learning_rate: 0.0010
Epoch 4/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.7524 - loss: 0.5146 - val_accuracy: 0.7778 - val_loss: 0.3653 - learning_rate: 0.0010
Epoch 5/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.8899 - loss: 0.2656 - val_accuracy: 0.8586 - val_loss: 0.3289 - learning_rate: 0.0010
Epoch 6/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.9776 - loss: 0.1160 - val_accuracy: 0.8889 - val_loss: 0.3719 - learning_rate: 0.0010
Epoch 7/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.9887 - loss: 0.0418 - val_acc

In [23]:
import random

def predict_random_review(model, tokenizer, reviews, max_text_len):
    # Виберіть випадковий відгук
    random_review = random.choice(reviews)
    print(f"Review: {random_review}")

    # Попередня обробка рецензії
    sequence = tokenizer.texts_to_sequences([random_review])
    padded_sequence = pad_sequences(sequence, maxlen=max_text_len)

    # Передбачити настрої
    prediction = model.predict(padded_sequence)

    # Визначити знак настрою
    sentiment = "Positive" if np.argmax(prediction) == 0 else "Negative"
    
    print(f"Predicted Sentiment: {sentiment}")

predict_random_review(model, tokenizer, comments, max_text_len)


Review: "The soundtrack failed to capture the magic of the wizarding world."
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
Predicted Sentiment: Negative
